Data Science Project: Phage Lysin Sequence Alignment Analysis 

With the ongoing rise of antibiotic resistance in bacteria, including challenging pathogens like Mycobacteria, finding alternative treatments for patients is becoming increasingly urgent. An emerging approach is the use of phage lysins to effectively target and eradicate specific bacterial infections. Lysins are enzymes produced by bacteriophages, they possess domains that adhere to bacterial cell wall components and trigger hydrolysis, ultimately resulting in the elimination of the host bacterium. Understanding the conservation and variability of amino acid sequences, especially in the context of Mycobacteria infections, is crucial for developing effective therapeutic lysin treatments. In this project, we will identify conserved regions, assess sequence similarity, and uncover evolutionary patterns to inform the development of targeted antibacterial therapies, specifically focusing on Mycobacteria skin and lung infections.

Data Wrangling

In [1]:
# Import Modules
import requests
import json
import re
import pandas as pd

Data collection

In [ ]:
# Fetch genes data from phagesdb API

# URL of the API endpoint
url = "https://phagesdb.org/api/genes/"

# Define the chunk size
chunk_size = 10000  # You can adjust this value based on your needs

# Initialize variables for pagination
page = 1
total_records = None

# Open the JSON file in write mode
with open('phagesdb_genes.json', 'w') as file:
    # Write an opening bracket to indicate the start of a JSON array
    file.write("[\n")
   
    # Fetch data in chunks until all records are retrieved
    while total_records is None or (page - 1) * chunk_size < total_records:
        # Make a request to the API with pagination parameters
        response = requests.get(url, params={'page': page, 'page_size': chunk_size})
       
        # Check if the request was successful
        if response.status_code == 200:
            # Convert the response to JSON format
            data = response.json()
           
            # Update the total number of records if it's not set yet
            if total_records is None:
                total_records = data['count']
           
            # Write the fetched data from the current page to the JSON file
            json.dump(data['results'], file)
           
            # Write a comma after each chunk except for the last one
            if (page - 1) * chunk_size + len(data['results']) < total_records:
                file.write(",\n")
           
            # Print a message to indicate progress
            print(f"Fetched {len(data['results'])} records (Chunk {page})")
        else:
            # Print an error message if the request was not successful
            print(f"Error fetching data: {response.status_code}")
            break
       
        # Increment the page number for the next request
        page += 1
   
    # Write a closing bracket to indicate the end of the JSON array
    file.write("\n]\n")

# Print a message to indicate successful completion
print("Data fetched and saved successfully!")


In [ ]:
#Combine outer lists generated from chunk

# Read the JSON file
with open('phagesdb_genes.json', 'r') as file:
    data = json.load(file)

# Combine the outer lists
combined_data = sum(data, [])

# Determine the number of lists in the final JSON data
num_lists = len(combined_data)
print("Number of lists in the final JSON data:", num_lists)

# Define the file path for the new JSON file
output_file = 'combined_data_output.json'

# Write the combined JSON data to the new file
with open(output_file, 'w') as file:
    json.dump(combined_data, file)

print("Combined JSON data saved to", output_file)

Data Definition and Cleaning

In [2]:
# Filter json file for only entries containing lysin in Notes field

# Load the combined_data_output.json file
input_file_path = 'combined_data_output.json'

with open(input_file_path, 'r') as input_file:
    data = json.load(input_file)

# Function to check if any form of "lysin" is present in the Notes
def contains_lysin(notes):
    lysin_pattern = re.compile(r'lysin', re.IGNORECASE)
    return bool(lysin_pattern.search(notes))

# Filter for dictionaries where the Notes key contains any form of "lysin"
lysin_dictionaries = []

print("Filtering dictionaries...")

for item in data:
    notes = item.get('Notes', '')
    if contains_lysin(notes):
        lysin_dictionaries.append(item)

print("Filtering complete.")

# Specify the path to the new JSON file
output_file_path = 'lysin_dictionaries.json'

# Write the filtered dictionaries to the new JSON file
print("Saving filtered dictionaries...")

with open(output_file_path, 'w') as output_file:
    json.dump(lysin_dictionaries, output_file)

num_saved_dictionaries = len(lysin_dictionaries)
print(f"Filtered dictionaries saved to '{output_file_path}'")
print(f"Number of dictionaries saved: {num_saved_dictionaries}")

# Load the combined_data_output.json file
input_file_path = 'combined_data_output.json'

with open(input_file_path, 'r') as input_file:
    combined_data_output = json.load(input_file)

Filtering dictionaries...
Filtering complete.
Saving filtered dictionaries...
Filtered dictionaries saved to 'lysin_dictionaries.json'
Number of dictionaries saved: 7182


In [3]:
# Un-nest PhageID dictionary in json file 

# Load the JSON data
with open('lysin_dictionaries.json', 'r') as file:
    data = json.load(file)

# Iterate through each gene entry
for gene_entry in data:
    # Unpack the PhageID dictionary
    phage_id_dict = gene_entry.pop('PhageID')
    for key, value in phage_id_dict.items():
        # Add each key-value pair to the outer dictionary
        gene_entry[key] = value

# Save the modified data to a new JSON file
output_file_path = 'lysin_unnested_data.json'
with open(output_file_path, 'w') as output_file:
    json.dump(data, output_file, indent=4)

print("Unnested data saved to:", output_file_path)


Unnested data saved to: lysin_unnested_data.json


In [4]:
# Read in json file as a pandas df
lysin_df = pd.read_json('lysin_unnested_data.json')
print(lysin_df.head())

        GeneID     phams  Start   Stop  Length  Name  \
0  20ES_CDS_10  [152741]   6442   7420     978  20ES   
1   20ES_CDS_8  [152812]   4689   5988    1299  20ES   
2   244_CDS_34  [152812]  29920  31423    1503   244   
3  32HC_CDS_36  [152814]  27404  28772    1368  32HC   
4  32HC_CDS_37    [3452]  28786  29938    1152  32HC   

                                         translation Orientation       Notes  \
0  MSLQVGSSGELVNRWIRVMKARFASYAGKLKEDGYFGLDDKAVQQE...           F  b'lysin B'   
1  MTAVITRKQAQWVHDMARARNGLPYAYGGAFTNDPKRSTDCSGLVL...           F  b'lysin A'   
2  MSVTRANVEATKRFIGERVGNPYVYGGALSPTNVHQGTDCSEVWQT...           F  b'lysin A'   
3  MPGSEIPRYWPLGAGRIVTSPFGPRSGGFHAGVDFGRNGGSAGMPV...           F  b'lysin A'   
4  MAWKQPQLTDPPMVSEEIGKLNRRLLLAYAANSRAVEAGVQLHDVF...           F  b'lysin B'   

  PhageID Accession     HostStrain Cluster  
0    20ES  KJ410132  Mycobacterium      A2  
1    20ES  KJ410132  Mycobacterium      A2  
2     244  DQ398041  Mycobacterium       E  
3 

In [5]:
# Investigate the DataFrame
print("DataFrame shape:", lysin_df.shape)

print(lysin_df.info())

DataFrame shape: (7182, 13)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7182 entries, 0 to 7181
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   GeneID       7182 non-null   object
 1   phams        7182 non-null   object
 2   Start        7182 non-null   int64 
 3   Stop         7182 non-null   int64 
 4   Length       7182 non-null   int64 
 5   Name         7182 non-null   object
 6   translation  7182 non-null   object
 7   Orientation  7182 non-null   object
 8   Notes        7182 non-null   object
 9   PhageID      7182 non-null   object
 10  Accession    7182 non-null   object
 11  HostStrain   7182 non-null   object
 12  Cluster      7117 non-null   object
dtypes: int64(3), object(10)
memory usage: 729.6+ KB
None


In [6]:
# Check for missing values in each column
missing_values = lysin_df.isna().sum()

# Print the missing values counts
print("Missing values counts:")
print(missing_values)


Missing values counts:
GeneID          0
phams           0
Start           0
Stop            0
Length          0
Name            0
translation     0
Orientation     0
Notes           0
PhageID         0
Accession       0
HostStrain      0
Cluster        65
dtype: int64


Cluster info is sometimes unknown and missing from this dataset. It will not affect the downsteam analysis. 

In [7]:
#Count duplicate values in each column

# Iterate over each column in the DataFrame
for column in lysin_df.columns:
    # Count the occurrences of each unique value in the column
    value_counts = lysin_df[column].value_counts()
    
    # Count the number of occurrences greater than 1 to find duplicates
    duplicate_count = sum(value_counts > 1)
    
    # Print column name and duplicate count
    print(f"Column: {column} | Duplicate Count: {duplicate_count}")

Column: GeneID | Duplicate Count: 0
Column: phams | Duplicate Count: 127
Column: Start | Duplicate Count: 1018
Column: Stop | Duplicate Count: 1037
Column: Length | Duplicate Count: 352
Column: Name | Duplicate Count: 2669
Column: translation | Duplicate Count: 987
Column: Orientation | Duplicate Count: 2
Column: Notes | Duplicate Count: 33
Column: PhageID | Duplicate Count: 2669
Column: Accession | Duplicate Count: 2664
Column: HostStrain | Duplicate Count: 11
Column: Cluster | Duplicate Count: 279


Most importantly each dictionary entry has a unique GeneID, it is not suprising that other Keys contain duplicate values. 

In [8]:
# Count and display Notes Column 

# Dictionary to store counts of different notes
notes_counts = {}

# Iterate through each entry in the "Notes" column
for note in lysin_df['Notes']:
    # Check if the value is not empty
    if note:
        # Update the count for the note value
        if note in notes_counts:
            notes_counts[note] += 1
        else:
            notes_counts[note] = 1

# Print all unique note values and their counts
for note, count in notes_counts.items():
    print(f"Notes: {note} | Count: {count}")


Notes: b'lysin B' | Count: 2517
Notes: b'lysin A' | Count: 2770
Notes: b'endolysin' | Count: 1007
Notes: b'putative lysin A' | Count: 8
Notes: b'putative lysin B' | Count: 6
Notes: b'LysM-like endolysin' | Count: 28
Notes: b'lysin A, protease M23 domain' | Count: 63
Notes: b'lysin' | Count: 28
Notes: b'lysin A, N-acetylmuramoyl-L-alanine amidase domain' | Count: 99
Notes: b'lysin A, L-Ala-D-Glu peptidase domain' | Count: 156
Notes: b'lysin A, glycosyl hydrolase domain' | Count: 213
Notes: b'lysin A, protease C39 domain' | Count: 93
Notes: b'lysin A, N-acetylmuramoyl-L-alanine amidase' | Count: 1
Notes: b'lysin A, M23 peptidase domain' | Count: 2
Notes: b'lysin A, amidase domain' | Count: 3
Notes: b'endolysin, L-Ala-D-Glu peptidase domain' | Count: 41
Notes: b'endolysin, N-acetylmuramoyl-L-alanine amidase domain' | Count: 45
Notes: b'endolysin, protease M23 domain' | Count: 7
Notes: b'lysin A, N-acetylmuramoyl-L-alanine' | Count: 2
Notes: b'gp24, lysin' | Count: 1
Notes: b'lysin A, L-al

The Notes field contains many different iterations of lysin. In order to proceed with analysis we will determine how many entries contain the following: lysin A, lysin B, endolysin, and other.

In [9]:
# Reclassify notes column and print Notes counts, Save df as json
def classify_notes(notes):
    if notes:
        notes_lower = notes.lower()
        if 'lysin a' in notes_lower or 'lysinA' in notes_lower:
            return 'lysin A'
        elif 'lysin b' in notes_lower or 'lysinB' in notes_lower:
            return 'lysin B'
        elif 'endolysin' in notes_lower:
            return 'endolysin'
        elif 'lysin' in notes_lower:
            return 'putative lysin'
    return notes

lysin_df['Notes'] = lysin_df['Notes'].apply(classify_notes)

# Count unique values in the 'Notes' column
notes_counts = lysin_df['Notes'].value_counts()

# Print all unique note values and their counts
print("Unique note values and their counts:")
print(notes_counts)

# Calculate the total count
total_count = notes_counts.sum()

# Print the total count
print("Total count after modifications:", total_count)

# Specify the path to save the final JSON file
final_json_file_path = "final_lysin_data.json"

# Save the modified DataFrame as a JSON file
lysin_df.to_json(final_json_file_path, orient='records', indent=4)

print("Final JSON file saved successfully!")

Unique note values and their counts:
Notes
lysin A           3457
lysin B           2527
endolysin         1147
putative lysin      51
Name: count, dtype: int64
Total count after modifications: 7182
Final JSON file saved successfully!


json file is now ready for further analysis. 